- Notebook to generate sentence embeddings using fine tuned BERT
- approach 1. adding last four layers of BERT
- approach 2. concatenate last four layers of BERT and take mean of these layers

In [ ]:
!pip install transformers
# pip install tensorflow-text

     |████████████████████████████████| 4.0 MB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 16.6 MB/s 
     |████████████████████████████████| 596 kB 29.0 MB/s 
     |████████████████████████████████| 77 kB 5.6 MB/s 
     |████████████████████████████████| 895 kB 35.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import torch
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import (BertForSequenceClassification, BertTokenizer, AdamW, BertConfig, get_linear_schedule_with_warmup)
from transformers import AutoTokenizer, AutoModel
from sklearn import preprocessing
from torch.utils.data import TensorDataset, random_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
from pandas.core.common import SettingWithCopyWarning
import operator
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [ ]:
if torch.cuda.is_available():        
    device = torch.device("cuda")
    print('GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('CPU is exist.')

GPU: Tesla K80


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased") # 'bert-base-uncased' original

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 2,                 
    output_attentions = False, 
    output_hidden_states = True,  # originally False,
)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
optimizer = AdamW(model.parameters(),lr = 2e-5, eps = 1e-8)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/AI Projects and Dataset/Health ML/IMDB Experiments/IMDB_chunked.csv")

In [ ]:
df.shape

(54147, 2)

In [ ]:
if torch.cuda.is_available():
    map_location=lambda storage, loc: storage.cuda()
else:
    map_location='cpu'

In [ ]:
output_model = '/content/drive/MyDrive/AI Projects and Dataset/Health ML/bert_fined_tune_chunked_IMDB.pth'

In [ ]:
checkpoint = torch.load(output_model, map_location=map_location)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
torch.cuda.empty_cache()

In [ ]:
texts = df.TEXT
labels = df.label

In [ ]:
# Tokenize all of the sequences and map the tokens to thier IDs.
input_ids_train = []
attention_masks_train = []

# For every sequences
for seq in texts:
    encoded_dict = tokenizer.encode_plus(
                        seq,                             
                        add_special_tokens = True,      
                        max_length = 256,              
                        padding = 'max_length',         
                        truncation=True,                
                        return_attention_mask = True,  
                        return_tensors = 'pt',          
                   )
    
    input_ids_train.append(encoded_dict['input_ids'])
    attention_masks_train.append(encoded_dict['attention_mask']) 
    
input_ids_train = torch.cat(input_ids_train, dim=0)
attention_masks_train = torch.cat(attention_masks_train, dim=0)
train_label = torch.tensor(labels.values)

In [ ]:
input_ids_train.shape, attention_masks_train.shape, train_label.shape

(torch.Size([54147, 256]), torch.Size([54147, 256]), torch.Size([54147]))

In [ ]:
df.shape, texts.shape, labels.shape

((54147, 2), (54147,), (54147,))

In [ ]:
dataset = TensorDataset(input_ids_train, attention_masks_train, train_label) 
prediction_sampler = SequentialSampler(dataset)
prediction_dataloader = DataLoader(dataset, sampler=prediction_sampler, batch_size=1)

In [ ]:
len(prediction_dataloader), len(prediction_sampler), len(dataset)

(54147, 54147, 54147)

In [ ]:
type(prediction_dataloader)

torch.utils.data.dataloader.DataLoader

In [ ]:
encoded_dict_temp = tokenizer.encode_plus(
                    df.TEXT[0],                             
                    add_special_tokens = True,      
                    max_length = 256,              
                    padding = 'max_length',         
                    truncation=True,                
                    return_attention_mask = True,  
                    return_tensors = 'pt',          
                )

In [ ]:
encoded_dict_temp = encoded_dict_temp.to(device)

In [ ]:
with torch.no_grad():
        outputs = model(encoded_dict_temp['input_ids'], token_type_ids=None, attention_mask=encoded_dict_temp['attention_mask'])

In [ ]:
hidden_states = outputs['hidden_states']

## Sentence Embeddings From BERT

In [ ]:
lst = torch.tensor([1,2,3,0,2,1,0])

In [ ]:
torch.sum(lst[-4])

tensor(0)

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
%%time
print('Predicting labels for {:,} test sentences'.format(len(input_ids_train)))

predictions , true_labels, embeddings = [], [], []

for batch in prediction_dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    
    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    hidden_states = outputs['hidden_states']
    last_four_layers = hidden_states[-4]
    last_four_layers_sum = torch.sum(last_four_layers, dim=0)
    last_four_layers_mean = torch.mean(last_four_layers_sum, dim=0)
    sentence_embedding_last_four = last_four_layers_mean.detach().cpu().numpy() 
    embeddings.append(sentence_embedding_last_four)
    logits = outputs[0]
    logits = logits.detach().cpu().numpy()    
    label_ids = b_labels.to('cpu').numpy()
    predictions.append(logits)
    true_labels.append(label_ids)
    

Predicting labels for 54,147 test sentences
CPU times: user 39min 55s, sys: 4.68 s, total: 40min
Wall time: 39min 52s


In [ ]:
len(embeddings)

54147

In [ ]:
# true_labels = labels.tolist()

In [ ]:
len(true_labels)

54147

In [ ]:
from sklearn.metrics import balanced_accuracy_score

In [ ]:
result_label, result_prob, result_logits = [], [], []

for j in range(len(predictions)):
    for i in range(len(predictions[j])):      
        index, value = max(enumerate(predictions[j][i]), key=operator.itemgetter(1))
        result_label.append(index)
        result_prob.append(value)
        result_logits.append(predictions[j][i])

In [ ]:
df['embeddings'] = embeddings

In [ ]:
df.head()

,TEXT,label,embeddings
0,one review ha mention watch 1 Oz episod youll ...,1,"[-0.08414811, 0.054563925, 0.6278188, -0.11547..."
1,wonder littl product film techniqu veri unassu...,1,"[-0.16437486, -0.23341937, 0.95019126, 0.03641..."
2,thought thi wa wonder way spend time hot summe...,1,"[-0.05472275, -0.32223767, 0.93231654, -0.1764..."
3,basic famili littl boy jake think zombi hi clo...,0,"[-0.26546022, -0.2735378, 0.825652, -0.1844702..."
4,petter mattei love time money visual stun film...,1,"[-0.06426333, -0.06923385, 1.0653114, -0.09120..."


In [ ]:
df.to_csv("IMDB_BERT_EMBEDDINGS_CHUNKED.csv", index = False)

In [ ]:
def build_numerical_dataset(features, df_kms:pd.DataFrame, isCategorical= False, num_classes = 4):
  columns = np.arange(0,features)
  df_temp = df_kms.copy()
  for x in range(0,df_temp.shape[0]):
    df_temp.embeddings[x] = df_temp.embeddings[x]
  print("Input dataframe shape:", df_temp.shape)
  df_split = pd.DataFrame(df_temp["embeddings"].to_list(), columns=columns)
  df_split.iloc[:1, :]
  df_split['label'] = df_kms.label
  print("Output dataframe shape:", df_split.shape)
  return df_split

In [ ]:
df_kms = build_numerical_dataset(768, df, isCategorical= False, num_classes = 2)

Input dataframe shape: (54147, 3)
Output dataframe shape: (54147, 769)


In [ ]:
df_kms.to_csv("BERT_EMBEDDINGS_IMDB_CHUNKED.csv", index = False)

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
df_kms_copy = df_kms.copy()

In [ ]:
Y = df_kms_copy.label

In [ ]:
df_kms_copy.drop(columns={'label'}, inplace = True)

In [ ]:
X = df_kms_copy
print(Y.shape, X.shape)

(54147,) (54147, 768)


In [ ]:
kmeans = KMeans(n_clusters=2, random_state=0)
kmeans.fit(X)

KMeans(n_clusters=2, random_state=0)

In [ ]:
pred = kmeans.labels_

In [ ]:
Y = df_kms.label

In [ ]:
correct_labels = sum(Y == pred)
print("Result: %d out of %d samples were correctly labeled." % (correct_labels, len(Y)))
print('Accuracy score:', np.round((correct_labels/float(len(Y)))*100, 2)," %")

Result: 26723 out of 54147 samples were correctly labeled.
Accuracy score: 49.35  %


In [ ]:
def cnn(df:pd.DataFrame, DataName: str, EPOCHS:int):
  print(DataName, ":", df.shape)
  Y = df.label
  X = df.drop(columns=['label'])
  Y = to_categorical(Y, 4)
  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size= 0.3)
  model = Sequential()
  model.add(Conv1D(512, 4, activation='relu', input_shape = (X_train.shape[1],1)))
  model.add(BatchNormalization()) 
  model.add(Dense(512, activation='relu'))
  # model.add(Dropout(0.5)) 
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(256, activation='relu'))
  model.add(Dense(128, activation='relu'))
  model.add(MaxPooling1D(2))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dense(2, activation='softmax'))
  model.compile(loss="categorical_crossentropy", optimizer = 'adam', metrics=["accuracy"])

  history = model.fit(X_train, Y_train,validation_split=0.25, epochs = EPOCHS, batch_size=32, shuffle=True, verbose=1)  
  predictions = model.predict(X_test)
  Y_pred = predictions.argmax(axis=1)
  Y_test_max = Y_test.argmax(axis=1)
  auc = roc_auc_score(Y_test_max, predictions, multi_class='ovr', average='macro')
  accuracy = np.round(accuracy_score(Y_test_max, Y_pred)*100,4)
  bal_acc = np.round(balanced_accuracy_score(Y_test_max, Y_pred)*100,4)
  recall = np.round(recall_score(Y_test_max, Y_pred, average='micro')*100,4)
  f1_weighted = np.round(f1_score(Y_test_max, Y_pred, average='weighted')*100,4)
  f1_average = np.round(f1_score(Y_test_max, Y_pred, average='micro')*100,4)
  precision = np.round(precision_score(Y_test_max, Y_pred, average='micro')*100,4)
  result = ["CNN", accuracy, bal_acc, recall,f1_weighted, f1_average, precision]
 
  cm = confusion_matrix(Y_test_max, Y_pred)
  plt.figure(figsize = (7,7))
  sns.heatmap(cm, annot=True)
  plt.show()
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('Model Accuracy for %s' % DataName)
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['train', 'val'], loc='upper left')
  plt.show()
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model loss for %s' % DataName)
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'val'], loc='upper left')
  plt.show()
  return result

In [ ]:
from nltk.tokenize.toktok import ToktokTokenizer
from prettytable import PrettyTable
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import roc_curve,auc
import pandas as pd
from tqdm import tqdm
import numpy as np
import os
import time
from nltk.tokenize import word_tokenize
from sklearn.manifold import TSNE
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm
tqdm.pandas()
import keras
from keras.utils import np_utils
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from nltk import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.utils.np_utils import to_categorical
from keras.layers.core import Dense, Dropout, Activation
import tensorflow as tf
# from tensorflow.keras import backend as K
from keras.layers import Embedding
from keras.layers import Input, Flatten
from keras.layers import Dense, Dropout, Conv1D, MaxPool1D, GlobalMaxPool1D, Embedding, Activation
from keras.layers import Dense, Input, LSTM, SimpleRNN, Embedding, Dropout, SpatialDropout1D, Activation, Conv1D,GRU
from keras.layers import Conv1D, Bidirectional, GlobalMaxPool1D, MaxPooling1D, BatchNormalization, Add, Flatten
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from sklearn.cluster import KMeans
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import gensim
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [ ]:
def classification_metrics(test_labels, predicted_labels, model_name):
  print()
  accuracy = np.round(accuracy_score(test_labels, predicted_labels)*100,4)
  bal_acc = np.round(balanced_accuracy_score(test_labels, predicted_labels)*100,4)
  recall = np.round(recall_score(test_labels, predicted_labels, average='micro')*100,4)
  f1_weighted = np.round(f1_score(test_labels, predicted_labels, average='weighted')*100,4)
  f1_average = np.round(f1_score(test_labels, predicted_labels, average='micro')*100,4)
  precision = np.round(precision_score(test_labels, predicted_labels, average='micro')*100,4)
  result = [model_name.upper(), accuracy, bal_acc, recall,f1_weighted, f1_average, precision]
  return result

In [ ]:
def baseline_models_experimental(df_experiments: pd.DataFrame, model_type: str):
  df_temp = df_experiments.copy()
  Y = df_temp.label
  df_temp.drop(columns={'label'}, inplace = True)
  X = df_temp
  x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size= 0.3)
  result_Final_exp = []
  def Random_Forest_EXP(x_train, x_test, y_train, y_test):
    forest = RandomForestClassifier()
    forest = forest.fit(x_train, y_train)
    result = forest.predict(x_test)
    result_Random_Forest_EXP = classification_metrics(y_test, result, "Random Forest EXP")
    result_Final_exp.append(result_Random_Forest_EXP)

  def Decision_Tree_EXP(x_train, x_test, y_train, y_test):
    modelDTC = DecisionTreeClassifier()
    modelLR = modelDTC.fit(x_train, y_train)
    result = modelLR.predict(x_test)
    result_DT_EXP = classification_metrics(y_test, result, "Decision Tree EXP")
    result_Final_exp.append(result_DT_EXP)

  def KNN_EXP(x_train, x_test, y_train, y_test):
    modelKNN = KNeighborsClassifier(n_neighbors=2)
    modelKNN = modelKNN.fit(x_train, y_train)
    result = modelKNN.predict(x_test)
    classes_x=result
    result_KNN_EXP = classification_metrics(y_test, classes_x, "K Nearest Neighbors EXP")
    result_Final_exp.append(result_KNN_EXP)

  def KMEANS_EXP(X: pd.DataFrame, Y: pd.DataFrame):

    kmeans = KMeans(n_clusters=2, random_state=0, max_iter=1000, algorithm = 'auto')
    kmeans.fit(X)
    pred = kmeans.labels_
    correct_labels = sum(Y == pred)
    # print("Result: %d out of %d samples were correctly labeled." % (correct_labels, len(Y)), end="\n\n")
    result_KMeans_exp = classification_metrics(Y, pred, "K Means EXP")
    result_Final_exp.append(result_KMeans_exp)

  
  Random_Forest_EXP(x_train, x_test, y_train, y_test)
  Decision_Tree_EXP(x_train, x_test, y_train, y_test)
  # KNN_EXP(x_train, x_test, y_train, y_test)
  KMEANS_EXP(X, Y)
  # result_cnn = cnn(df_experiments, "CNN EXP", 10)
  # result_Final_exp.append(result_cnn)
  Pretty_Table = PrettyTable(["Model" , "Accuracy", "Balanced Accuracy", "Recall", "F1-Weighted", "F1-Average", "Precision"])
  Pretty_Table.title = model_type.upper()
  for x in result_Final_exp:
    Pretty_Table.add_row(x)

  print(Pretty_Table, end = "\n")


In [ ]:
baseline_models_experimental(df_kms, "BERT 768 Dimension Last Four Layers Means")




+---------------------------------------------------------------------------------------------------+
|                             BERT 768 DIMENSION LAST FOUR LAYERS MEANS                             |
+-------------------+----------+-------------------+---------+-------------+------------+-----------+
|       Model       | Accuracy | Balanced Accuracy |  Recall | F1-Weighted | F1-Average | Precision |
+-------------------+----------+-------------------+---------+-------------+------------+-----------+
| RANDOM FOREST EXP | 90.5817  |      90.5798      | 90.5817 |   90.5813   |  90.5817   |  90.5817  |
| DECISION TREE EXP | 83.9212  |      83.9206      | 83.9212 |   83.9211   |  83.9212   |  83.9212  |
|    K MEANS EXP    | 49.3527  |      49.3967      | 49.3527 |   49.1964   |  49.3527   |  49.3527  |
+-------------------+----------+-------------------+---------+-------------+------------+-----------+
